<a href="https://colab.research.google.com/github/Decoding-Data-Science/CommunityWorkshops/blob/main/WeatherAssistant_Jul17/Weather_assistant_8th_march_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai==0.28 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [ ]:
!pip install gradio

In [4]:
import gradio as gr

In [5]:
import openai

In [6]:
import os
from google.colab import userdata

# Retrieve API keys from Colab's secure storage
openai_api_key = userdata.get("openai")

# Set them as environment variables

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key

In [7]:
import requests

def get_current_weather(location, unit='celsius'):
    weather_api_key = "a7b109315b6ced6a9cd5177bdb98ca82"
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(base_url)
    data = response.json()

    weather_description = data['weather'][0]['description']
    return {
        "location": location,
        "temperature": data['main']['temp'],
        "weather": weather_description
    }

In [8]:
print(get_current_weather("Dubai"))

{'location': 'Dubai', 'temperature': 31.96, 'weather': 'clear sky'}


In [9]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, e.g. San Francisco"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [10]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city, e.g. San Francisco'},
    'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
   'required': ['location']}}]

In [11]:
#Function Definition and Initial Message Handling
def weather_chat(user_message):
    messages=[]
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": "You are a weather bot . Answer only in Celsius."})



        # Sending Initial Message to OpenAI
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0,
            messages=messages,
            functions=functions
        )

    #Handling Function Calls and Fetching Weather Data
    try:
        function_call = response['choices'][0]['message']['function_call']
        arguments = eval(function_call['arguments'])

        # Fetch weather data using the extracted arguments
        weather_data = get_current_weather(arguments['location'])

        # Append the function call and weather data to the messages
        messages.append({"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": str(arguments)}})
        messages.append({"role": "function", "name": "get_current_weather", "content": str(weather_data)})

#magic of llm
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        return response['choices'][0]['message']['content']
    except Exception as e:
        return "I'm here to provide weather updates. Please ask me questions related to weather."



In [12]:
weather_chat("is it hot in DUbaii")

'Yes, it is currently 31.96°C in Dubai with clear skies.'

In [ ]:
#qc
weather_chat("كم درجة الحرارة في الشارقة؟ ")

'درجة الحرارة في الشارقة هي حالياً حوالي 38.16 درجة مئوية مع سماء صافية.'

In [ ]:
# Define Gradio interface
iface = gr.Interface(
    fn=weather_chat,
    inputs=gr.Textbox(label="Weather Queries"),
    outputs=gr.Textbox(label="Weather Updates"),
    title = "Weather Bot",
    description = "Ask me anything about weather!"
)

# Launch the Gradio interface
iface.launch(share="True")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0e23e7bd17e70bcfc3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
pip install transformers

In [ ]:
pip install mistral_inference

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

GatedRepoError: 401 Client Error. (Request ID: Root=1-66eeb497-7ab8278a1b8f41f64089594f;01a55d68-caab-41cb-a12d-9eb6d45b25bc)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3/resolve/e0bc86c23ce5aae1db576c8cca6f06f1f73af2db/consolidated.safetensors.
Access to model mistralai/Mistral-7B-Instruct-v0.3 is restricted. You must have access to it and be authenticated to access it. Please log in.